# Lecture 6.2: More on Relational Data and SQL
<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand how to [filter data using joins](#Filtering-joins)
* Know to execute basic [SQL commands](#SQL-Commands)
</div>

This lecture corresponds to Chapter 13 of your textbook.

In [1]:
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Filtering joins

Filtering joins allow us to filter rows on one table based on their presence or absence in another table. 

In [2]:
top5 <- count(flights, carrier) %>% top_n(5) %>% print
top5$carrier
flights_top5 = filter(flights, carrier %in% top5$carrier) %>% print

Selecting by n


# A tibble: 5 x 2
  carrier     n
  <chr>   <int>
1 AA      32729
2 B6      54635
3 DL      48110
4 EV      54173
5 UA      58665


[1] "AA" "B6" "DL" "EV" "UA"

# A tibble: 248,312 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753            745
# … with 248,302 more rows,

### Question: 
What if we wanted to filter based on more than one variable? For example, based on busiest days of the year:

In [2]:
top7_days  <- count(flights,year, month, day)  %>% top_n(7) %>% arrange(n) %>% print

Selecting by n



# A tibble: 7 x 4
   year month   day     n
  <int> <int> <int> <int>
1  2013     7    18  1003
2  2013     7    25  1003
3  2013     7     8  1004
4  2013     7    10  1004
5  2013    12     2  1004
6  2013     7    11  1006
7  2013    11    27  1014


One option would be to construct a new string variable which we could then match using the same technique:

In [3]:
top7_days_ymd <- unite(top7_days, ymd, year:day,sep="-") %>% print
flights_ymd <- unite(flights, ymd, year:day,sep="-") %>% print
filter(flights_ymd,ymd %in% top7_days_ymd$ymd) %>% print

# A tibble: 7 x 2
  ymd            n
  <chr>      <int>
1 2013-7-18   1003
2 2013-7-25   1003
3 2013-7-8    1004
4 2013-7-10   1004
5 2013-12-2   1004
6 2013-7-11   1006
7 2013-11-27  1014
# A tibble: 336,776 x 17
   ymd   dep_time sched_dep_time dep_delay arr_time sched_arr_time arr_delay
   <chr>    <int>          <int>     <dbl>    <int>          <int>     <dbl>
 1 2013…      517            515         2      830            819        11
 2 2013…      533            529         4      850            830        20
 3 2013…      542            540         2      923            850        33
 4 2013…      544            545        -1     1004           1022       -18
 5 2013…      554            600        -6      812            837       -25
 6 2013…      554            558        -4      740            728        12
 7 2013…      555            600        -5      913            854        19
 8 2013…      557            600        -3      709            723       -14
 9 2013…      55

But this is cumbersome and will also be slow if you are matching to a large list of choices.

### Semi-join
A better option is to use the built-in command `semi_join`. `semi_join(x, y)` keeps all the observations in `x` that are also in `y`.
![semi join](http://r4ds.had.co.nz/diagrams/join-semi.png)

`semi_join()` takes the same `key=` options as the other join commands.

**Question:** Now let us try to create a table with all flights in the 7 busiest days of the year.

In [4]:
semi_join(flights,top7_days, key=c("year","month","day")) %>% print

Joining, by = c("year", "month", "day")



# A tibble: 7,038 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013    11    27       28           1930       298      129           2049
 2  2013    11    27      505            500         5      703            651
 3  2013    11    27      514            515        -1      743            808
 4  2013    11    27      535            540        -5      907            850
 5  2013    11    27      540            545        -5      815            835
 6  2013    11    27      551            600        -9      920            854
 7  2013    11    27      551            600        -9      918            849
 8  2013    11    27      554            600        -6      719            719
 9  2013    11    27      554            600        -6      707            717
10  2013    11    27      554            600        -6      907            846
# … with 7,028 more rows, and

### Exercise
Of the 336,776 flights in `flights`, what % of them departed on one of the 20 busiest days of the year?

In [5]:
top20_days <- flights %>% count(year,month,day) %>% top_n(20) %>% print
semi_join(flights,top20_days, key=c("year","month","day"), by=c("year","month","day")) %>% nrow 

20046/nrow(flights)

Selecting by n



# A tibble: 20 x 4
    year month   day     n
   <int> <int> <int> <int>
 1  2013     7     8  1004
 2  2013     7     9  1001
 3  2013     7    10  1004
 4  2013     7    11  1006
 5  2013     7    12  1002
 6  2013     7    17  1001
 7  2013     7    18  1003
 8  2013     7    22  1000
 9  2013     7    24  1000
10  2013     7    25  1003
11  2013     7    31  1001
12  2013     8     1  1000
13  2013     8     5  1000
14  2013     8     7  1001
15  2013     8     8  1001
16  2013     8    12  1001
17  2013     8    15  1000
18  2013    11    21  1000
19  2013    11    27  1014
20  2013    12     2  1004


[1] 20046

[1] 0.05952324

### Anti-join
The opposite of a semi-join is an anti-join:
![anti-join](http://r4ds.had.co.nz/diagrams/join-anti.png)

### Exercise
<table><tr><th>Beginner</th><th>Advanced</th></tr>
    <tr>
        <td>How many flights were on planes which are <i>not</i> found in the <code>planes</code> table?</td>
        <td>How many planes flew to destinations which are <i>not</i> found in the <code>airports</code> table?</td>
    </tr>
    </table>

In [6]:
anti_join(flights,planes,by="tailnum") %>%nrow

anti_join(flights,airports,by=c("dest"="faa")) %>% distinct(tailnum) %>% print

[1] 52606

# A tibble: 804 x 1
   tailnum
   <chr>  
 1 N804JB 
 2 N794JB 
 3 N3BAAA 
 4 N77296 
 5 N651JB 
 6 N527JB 
 7 N900PC 
 8 N5FSAA 
 9 N5EXAA 
10 N24715 
# … with 794 more rows


Anti-joins are useful for finding mismatches in joins. Last lecture we saw that some 50k entries in `flights` have `tailnum`s that do not exist in the `planes` table. To find out more about these flights we could use an anti-join to select only those rows:

In [23]:
missing_planes <- anti_join(flights, planes, by="tailnum") %>% 
                 filter(!is.na(tailnum)) %>% print

# A tibble: 50,094 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      558            600        -2      753            745
 2  2013     1     1      559            600        -1      941            910
 3  2013     1     1      600            600         0      837            825
 4  2013     1     1      602            605        -3      821            805
 5  2013     1     1      608            600         8      807            735
 6  2013     1     1      611            600        11      945            931
 7  2013     1     1      623            610        13      920            915
 8  2013     1     1      624            630        -6      840            830
 9  2013     1     1      628            630        -2     1137           1140
10  2013     1     1      629            630        -1      824            810
# … with 50,084 more rows, a

Do we see any pattern in this table that could explain the missing tail numbers?

In [24]:
missing_planes %>% sample_n(100) %>% print(width = Inf)

# A tibble: 100 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     8    11      621            630        -9      737            755
 2  2013    10    19     1157           1200        -3     1527           1515
 3  2013     7    27     1437           1445        -8     1703           1740
 4  2013    12    23      652            655        -3      935            940
 5  2013    12    23     1019           1005        14     1205           1155
 6  2013     4    25     1130           1100        30     1230           1210
 7  2013     9    10     1046           1050        -4     1243           1250
 8  2013     5     5     2052           2105       -13     2227           2305
 9  2013     2    14     2039           2045        -6     2159           2225
10  2013     9    20      730            735        -5     1018           1035
   arr_delay carrier flight tai

Interestingly, the top 5 most frequent `tailnum`s in the overall data are almost the same:

In [25]:
flights %>% drop_na(tailnum) %>% left_join(planes, by = "tailnum") %>% 
    count(tailnum, model) %>% top_n(5)

Selecting by n


tailnum,model,n
N711MQ,G1159B,486
N713MQ,NA,483
N722MQ,NA,513
N723MQ,NA,507
N725MQ,NA,575


In [26]:
planes %>% filter(tailnum == "N711MQ")

tailnum,year,type,manufacturer,model,engines,seats,speed,engine
N711MQ,1976,Fixed wing multi engine,GULFSTREAM AEROSPACE,G1159B,2,22,NA,Turbo-jet


The one plane that is represented is N711M. You can [look it up](https://flightaware.com/resources/registration/N711MQ) in an online database and find out it's a private jet. My guess is the others are private jets too.